In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch import nn,optim
from torch.utils.data import Dataset, DataLoader

from generic_data import GenericDataset
from SimpleNNs import TwoNet

In [2]:
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
def target(x):
    shift = np.pi / 8.0 # np.pi / 4.0
    #f1 = (np.sin(8 * x) / (4 * np.cos(2 * x))).reshape(-1,1)
    f1 = (np.sin(8 * (x+ 2*shift)) / (4 * np.cos(2 * (x + 2*shift)))).reshape(-1,1)
    f2 = (np.sin(8 * (x+shift)) / (4 * np.cos(2 * (x+shift)))).reshape(-1,1)
    return np.hstack((f1,f2))

In [4]:
# define range
low = -2
high = 2

In [5]:
# generate test set
Xtest = np.linspace(low,high,100)#np.sort(np.random.uniform(low, high, 100))
Ytest = target(Xtest)

In [6]:
test_dataset = GenericDataset(Xtest.reshape(-1,1), Ytest)

In [7]:
subfolder = "multi_random_init0"
paths = [f'./models/{subfolder}/model_{i:02d}.pt' for i in range(20)]

In [8]:
all_preds = []
all_labels = []
all_mses = []

In [9]:
device="cuda"

In [10]:
test_loader = DataLoader(dataset=test_dataset,batch_size=64,shuffle=False)

In [11]:
num_models = len(paths)
cnt = 1

for path in paths:
    model = torch.load(path)
    model.to(device)
    model.eval()
    
    preds = []
    labels = []
    
    for batch_vec, batch_labels in iter(test_loader):
        batch_vec = batch_vec.to(device)
        batch_outputs = model(batch_vec)
        batch_preds = batch_outputs.cpu().detach().numpy()
        batch_labels = batch_labels.numpy()
        preds.append(batch_preds)
        labels.append(batch_labels)
    
    preds = np.concatenate(preds)
    labels = np.concatenate(labels)
        
    regression_mse_f1 = np.square(np.subtract(preds[:,0], labels[:,0])).mean()
    regression_mse_f2 = np.square(np.subtract(preds[:,1], labels[:,1])).mean()
    
    mse = np.array([regression_mse_f1, regression_mse_f2])
    
    print(path)
    print(f'f1 MSE: {regression_mse_f1:.4f}, f2 MSE: {regression_mse_f2:.4f}. Model {cnt}/{num_models}')
    
    all_preds.append(preds)
    all_labels.append(labels)
    all_mses.append(mse)
    
    cnt += 1

./models/multi_random_init0/model_00.pt
f1 MSE: 0.0425, f2 MSE: 0.0385. Model 1/20
./models/multi_random_init0/model_01.pt
f1 MSE: 0.0016, f2 MSE: 0.0026. Model 2/20
./models/multi_random_init0/model_02.pt
f1 MSE: 0.0020, f2 MSE: 0.0013. Model 3/20
./models/multi_random_init0/model_03.pt
f1 MSE: 0.0028, f2 MSE: 0.0015. Model 4/20
./models/multi_random_init0/model_04.pt
f1 MSE: 0.0019, f2 MSE: 0.0033. Model 5/20
./models/multi_random_init0/model_05.pt
f1 MSE: 0.0020, f2 MSE: 0.0025. Model 6/20
./models/multi_random_init0/model_06.pt
f1 MSE: 0.0072, f2 MSE: 0.0123. Model 7/20
./models/multi_random_init0/model_07.pt
f1 MSE: 0.0031, f2 MSE: 0.0019. Model 8/20
./models/multi_random_init0/model_08.pt
f1 MSE: 0.0025, f2 MSE: 0.0024. Model 9/20
./models/multi_random_init0/model_09.pt
f1 MSE: 0.0020, f2 MSE: 0.0016. Model 10/20
./models/multi_random_init0/model_10.pt
f1 MSE: 0.0025, f2 MSE: 0.0042. Model 11/20
./models/multi_random_init0/model_11.pt
f1 MSE: 0.0025, f2 MSE: 0.0027. Model 12/20
.

In [12]:
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_mses = np.array(all_mses)

In [13]:
np.save(f'./models/{subfolder}/xs', Xtest)
np.save(f'./models/{subfolder}/predictions', all_preds)
np.save(f'./models/{subfolder}/true_labels', all_labels[0])
np.save(f'./models/{subfolder}/mses', all_mses)